# Project: The Battle of Neighborhoods

## Introduction
Salamanca is a Spanish university city. Due to the large number of students, most of the services available in the city are oriented to meet the needs of university students.
Above all, there are many food establishments.
However, food from establishments close to university areas does not offer good quality food and managers are not aware of creating businesses that respect sustainability and provide ecological products.
That's why Pepe, a citizen of Salamanca wants to embark on a sustainable business that sells organic products and food made from them.


## Objetive

The aim of this work is to provide Pepe with a tool that, using machine learning techniques, provides Pepe with information about the services available in the city so that he can choose the area that best suits him to start his business.


## Data

In order to solve the problem we use the data provided by the web foursquare for the coordinates of the city of Salamanca.

In the first place the coordinates of the city centre will be obtained.

Then, a query in the Foursquare website is done to obtain the top sites in Salamanca with their geographical coordinates.


In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    cer

In [2]:
url='http://www.gpspostcode.com/zip-code/spain/salamanca/?tri=cp_asc'

df=pd.read_html(url, header=0)[0]

df.head(15)

,Unnamed: 0,Zip code,City,X,Y
0,NaN,37001,Salamanca,40.9682,-5.6643
1,NaN,37100,Ledesma,41.0867,-6.0012
2,NaN,37110,San pelayo de la guareña,41.1353,-5.8302
3,NaN,37110,Santiz,41.2055,-5.8975
4,NaN,37110,Aldearrodrigo,41.1100,-5.8077
5,NaN,37110,Torresmenudas,41.1021,-5.7834
6,NaN,37110,Zamayon,41.1494,-5.8306
7,NaN,37110,El arco,41.1104,-5.8245
8,NaN,37111,Palacino,41.1527,-5.9033
9,NaN,37111,Palacios del arzobispo,41.1653,-5.8905


In [3]:
df=df.drop(['Unnamed: 0'], axis=1)
df.head(10)

,Zip code,City,X,Y
0,37001,Salamanca,40.9682,-5.6643
1,37100,Ledesma,41.0867,-6.0012
2,37110,San pelayo de la guareña,41.1353,-5.8302
3,37110,Santiz,41.2055,-5.8975
4,37110,Aldearrodrigo,41.1100,-5.8077
5,37110,Torresmenudas,41.1021,-5.7834
6,37110,Zamayon,41.1494,-5.8306
7,37110,El arco,41.1104,-5.8245
8,37111,Palacino,41.1527,-5.9033
9,37111,Palacios del arzobispo,41.1653,-5.8905


In [4]:
df=df.rename(columns={'Zip code':'zipcode'})
df.head(10)

,zipcode,City,X,Y
0,37001,Salamanca,40.9682,-5.6643
1,37100,Ledesma,41.0867,-6.0012
2,37110,San pelayo de la guareña,41.1353,-5.8302
3,37110,Santiz,41.2055,-5.8975
4,37110,Aldearrodrigo,41.1100,-5.8077
5,37110,Torresmenudas,41.1021,-5.7834
6,37110,Zamayon,41.1494,-5.8306
7,37110,El arco,41.1104,-5.8245
8,37111,Palacino,41.1527,-5.9033
9,37111,Palacios del arzobispo,41.1653,-5.8905


In [5]:
address = 'Salamanca'
geolocator = Nominatim(user_agent="Salamanca")
location = geolocator.geocode(address)
latitude_sal = location.latitude
longitude_sal = location.longitude
print('The geograpical coordinates of Salamanca are {}, {}.'.format(latitude_sal, longitude_sal))

The geograpical coordinates of Salamanca are 40.9651572, -5.6640182.


In [6]:
map_sal = folium.Map(location=[latitude_sal, longitude_sal], zoom_start=20)

# add markers to map
for zipcode, City, X, Y in zip(df['zipcode'], df['City'], df['X'], df['Y']):
    label = '{}, {}'.format(zipcode,City)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude_sal, longitude_sal],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_sal)  
    
map_sal

In [7]:
CLIENT_ID = 'QIWCWGVQZAZS5KEKEJ3HP5ZACX1PJ3CISJKNHREH5CIVO2KI' # your Foursquare ID
CLIENT_SECRET = 'H4ISQZTNOL1LLFA3IOZ0VVB1TZYYEDZWPUYOU1SSF2AOCDW3' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: QIWCWGVQZAZS5KEKEJ3HP5ZACX1PJ3CISJKNHREH5CIVO2KI
CLIENT_SECRET:H4ISQZTNOL1LLFA3IOZ0VVB1TZYYEDZWPUYOU1SSF2AOCDW3


In [8]:
LIMIT = 200 # limit of number of venues returned by Foursquare API

radius = 1500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude_sal, 
    longitude_sal, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=QIWCWGVQZAZS5KEKEJ3HP5ZACX1PJ3CISJKNHREH5CIVO2KI&client_secret=H4ISQZTNOL1LLFA3IOZ0VVB1TZYYEDZWPUYOU1SSF2AOCDW3&v=20180605&ll=40.9651572,-5.6640182&radius=1500&limit=200'

In [9]:
results = requests.get(url).json()

In [10]:
results

{'meta': {'code': 200, 'requestId': '5dcd188b0be7b4001bfadbcb'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Salamanca',
  'headerFullLocation': 'Salamanca',
  'headerLocationGranularity': 'city',
  'totalResults': 165,
  'suggestedBounds': {'ne': {'lat': 40.978657213500014,
    'lng': -5.646173335139151},
   'sw': {'lat': 40.951657186499986, 'lng': -5.681863064860849}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b7f4c0ef964a520c32530e3',
       'name': 'Plaza Mayor',
       'location': {'address': 'Pl. Mayor',
        'lat': 40.965009280015074,
        'lng': -5.664117336273193,
        'distance': 18,
        'postalCode': '37002',
        'cc': 'ES',
        'neighborh

In [11]:

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [12]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.location.postalCode','venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues_def=nearby_venues
nearby_venues_def.head(10)

,postalCode,name,categories,lat,lng
0,37002,Plaza Mayor,Plaza,40.965009,-5.664117
1,37002,Cuzco Bodega,Tapas Restaurant,40.964170,-5.665567
2,NaN,Capitán Haddock,Cocktail Bar,40.966406,-5.664584
3,NaN,The Holy Cross,Pub,40.964728,-5.661910
4,37002,Doctrinos,Wine Bar,40.964728,-5.665949
5,37002,La Tagliatella,Italian Restaurant,40.964845,-5.666487
6,37008,Casa Vallejo,Spanish Restaurant,40.964066,-5.663336
7,37001,Revolutum Hostel,Hotel Bar,40.963764,-5.664323
8,NaN,iPan iVino,Wine Bar,40.963466,-5.664555
9,37008,La Hoja 21,Spanish Restaurant,40.961701,-5.664190


In [13]:
print('{} venues were returned by Foursquare.'.format(nearby_venues_def.shape[0]))

100 venues were returned by Foursquare.


In [14]:
nearby_venues_def.groupby('postalCode').count()


,name,categories,lat,lng
postalCode,,,,
37001,10,10,10,10
37002,22,22,22,22
37003,2,2,2,2
37004,2,2,2,2
37005,3,3,3,3
37007,2,2,2,2
37008,16,16,16,16
España,1,1,1,1


In [15]:
# one hot encoding
sal_onehot = pd.get_dummies(nearby_venues_def['categories'], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
sal_onehot['postalCode'] = nearby_venues['postalCode'] 

# move neighborhood column to the first column
fixed_columns = [sal_onehot.columns[-1]] + list(sal_onehot.columns[:-1])

sal_onehot = sal_onehot[fixed_columns]

#Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
sal_grouped = sal_onehot.groupby('postalCode').mean().reset_index()


sal_onehot.shape

(100, 42)

In [163]:
num_top_venues = 5

for hood in sal_grouped['postalCode']:
    print("----"+hood+"----")
    temp = sal_grouped[sal_grouped['postalCode'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----37001----
              venue  freq
0  Tapas Restaurant   0.3
1          Wine Bar   0.1
2        Restaurant   0.1
3         Gastropub   0.1
4              Park   0.1


----37002----
                      venue  freq
0          Tapas Restaurant  0.09
1                      Café  0.09
2  Mediterranean Restaurant  0.09
3                     Hotel  0.05
4              Gourmet Shop  0.05


----37003----
                venue  freq
0                Park   0.5
1              Museum   0.5
2           Hotel Bar   0.0
3           Irish Pub   0.0
4  Italian Restaurant   0.0


----37004----
                venue  freq
0               Plaza   0.5
1        Burger Joint   0.5
2                Park   0.0
3           Irish Pub   0.0
4  Italian Restaurant   0.0


----37005----
                 venue  freq
0                  Bar  0.25
1     Tapas Restaurant  0.25
2  Japanese Restaurant  0.25
3                Diner  0.25
4          Art Gallery  0.00


----37007----
                venue  freq
0       

In [16]:
sal_grouped

,postalCode,Art Gallery,Art Museum,Bakery,Bar,Bed & Breakfast,Brewery,Burger Joint,Café,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Quad,Concert Hall,Dessert Shop,Diner,Garden,Gastropub,Gourmet Shop,Historic Site,Hotel,Hotel Bar,Indie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Mediterranean Restaurant,Middle Eastern Restaurant,Monument / Landmark,Museum,Nightclub,Park,Plaza,Pub,Restaurant,River,Snack Place,Spanish Restaurant,Tapas Restaurant,Vegetarian / Vegan Restaurant,Wine Bar
0,37001,0.0,0.000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.1,0.1,0.000000,0.0,0.0000,0.000000,0.0,0.000000,0.0,0.1,0.000000,0.0,0.000000,0.1,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.100000,0.000000,0.000000,0.100000,0.0,0.0000,0.000000,0.300000,0.0000,0.100000
1,37002,0.0,0.000,0.0,0.045455,0.0,0.045455,0.0,0.045455,0.0,0.0,0.045455,0.0,0.0000,0.045455,0.0,0.000000,0.0,0.0,0.045455,0.0,0.045455,0.0,0.0,0.045455,0.045455,0.000000,0.090909,0.0,0.045455,0.0,0.045455,0.045455,0.045455,0.090909,0.045455,0.0,0.0000,0.045455,0.090909,0.0000,0.045455
2,37003,0.0,0.000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.5,0.000000,0.500000,0.000000,0.000000,0.000000,0.0,0.0000,0.000000,0.000000,0.0000,0.000000
3,37004,0.0,0.000,0.0,0.000000,0.0,0.000000,0.5,0.000000,0.0,0.0,0.000000,0.0,0.0000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.500000,0.000000,0.000000,0.0,0.0000,0.000000,0.000000,0.0000,0.000000
4,37005,0.0,0.000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0000,0.000000,0.0,0.333333,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.333333,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0000,0.000000,0.333333,0.0000,0.000000
5,37007,0.0,0.000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.5,0.0000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.500000,0.000000,0.000000,0.0,0.0000,0.000000,0.000000,0.0000,0.000000
6,37008,0.0,0.125,0.0,0.062500,0.0,0.000000,0.0,0.062500,0.0,0.0,0.000000,0.0,0.0625,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.250000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.062500,0.0,0.000000,0.000000,0.062500,0.000000,0.062500,0.0,0.0625,0.125000,0.000000,0.0625,0.000000
7,España,1.0,0.000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0000,0.000000,0.000000,0.0000,0.000000


In [17]:
 #set number of clusters
kclusters = 3

sal_grouped_clustering = sal_grouped.drop('postalCode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(sal_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 0, 2, 0, 2, 0, 1], dtype=int32)

In [18]:
# add clustering labels
#sal_merged = df_def.loc[df_def['Neighbourhood'].isin(neighborhoods_venues_sorted['Neighborhood'])]
sal_merged = nearby_venues_def.groupby('postalCode').count()
sal_merged['Cluster Labels'] = kmeans.labels_


sal_merged # check the last columns!

,name,categories,lat,lng,Cluster Labels
postalCode,,,,,
37001,10,10,10,10,0
37002,22,22,22,22,0
37003,2,2,2,2,0
37004,2,2,2,2,2
37005,3,3,3,3,0
37007,2,2,2,2,2
37008,16,16,16,16,0
España,1,1,1,1,1


In [19]:
nearby_venues_def.groupby('categories').count().index

Index(['Art Gallery', 'Art Museum', 'Bakery', 'Bar', 'Bed & Breakfast',
       'Brewery', 'Burger Joint', 'Café', 'Church', 'Clothing Store',
       'Cocktail Bar', 'Coffee Shop', 'College Quad', 'Concert Hall',
       'Dessert Shop', 'Diner', 'Garden', 'Gastropub', 'Gourmet Shop',
       'Historic Site', 'Hotel', 'Hotel Bar', 'Indie Theater', 'Irish Pub',
       'Italian Restaurant', 'Japanese Restaurant', 'Mediterranean Restaurant',
       'Middle Eastern Restaurant', 'Monument / Landmark', 'Museum',
       'Nightclub', 'Park', 'Plaza', 'Pub', 'Restaurant', 'River',
       'Snack Place', 'Spanish Restaurant', 'Tapas Restaurant',
       'Vegetarian / Vegan Restaurant', 'Wine Bar'],
      dtype='object', name='categories')

In [20]:
# create map
map_clusters = folium.Map(location=[latitude_sal, longitude_sal], zoom_start=10)

# set color scheme for the clusters
x = np.arange(8)
ys = [i + x + (i*x)**2 for i in range(8)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
print(rainbow)
postal = list(nearby_venues_def.groupby('postalCode').count().index)
print(postal)

# add markers to the map
markers_colors = []
for lat, lon, post, name in zip(nearby_venues_def['lat'], nearby_venues_def['lng'] , nearby_venues_def['postalCode'], nearby_venues_def['name']):
    label = folium.Popup(str(name) + ' category ' + str(post), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=3,
        popup=label,
        color='#386df9',
        fill=True,
        fill_color='#386df9',
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

['#8000ff', '#386df9', '#12c8e6', '#5af8c8', '#a4f89f', '#ecc86f', '#ff6d38', '#ff0000']
['37001', '37002', '37003', '37004', '37005', '37007', '37008', 'España']
